In [ ]:
! git clone git@github.com:amaranto/computer-vision-cards.git .

In [ ]:
! pip install -r requirements.txt

In [ ]:
!python datasetHandler.py

In [ ]:
!python augmentation.py

In [ ]:
!python datasetViewer.py